In [1]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/home/fabricio/Documentos/Projetos/tcc_predict_crypto/model_predict/.predict/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DAYS_PREDICT = 15

In [3]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_ada = yf.download("ADA-USD", data_init, date_today)

df_ada.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,0.025160,0.035060,0.025006,0.032053,0.032053,18716200
2017-11-10,0.032219,0.033348,0.026451,0.027119,0.027119,6766780
2017-11-11,0.026891,0.029659,0.025684,0.027437,0.027437,5532220
2017-11-12,0.027480,0.027952,0.022591,0.023977,0.023977,7280250
2017-11-13,0.024364,0.026300,0.023495,0.025808,0.025808,4419440


In [4]:
df_ada.reset_index(inplace=True)
df_ada.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1768,2022-09-12,0.509751,0.518862,0.497820,0.501546,0.501546,856809100
1769,2022-09-13,0.501495,0.507524,0.462914,0.468881,0.468881,1004511415
1770,2022-09-14,0.463912,0.485709,0.463049,0.481376,0.481376,737527013
1771,2022-09-15,0.481374,0.481406,0.463869,0.465644,0.465644,685227625
1772,2022-09-16,0.465703,0.474130,0.456019,0.474126,0.474126,558761609


In [5]:
df = df_ada[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [6]:
df.tail()

,ds,y
1768,2022-09-12,0.501546
1769,2022-09-13,0.468881
1770,2022-09-14,0.481376
1771,2022-09-15,0.465644
1772,2022-09-16,0.474126


In [7]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [8]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [9]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:33:21 - cmdstanpy - INFO - Chain [1] start processing
00:33:22 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
1783,2022-09-27
1784,2022-09-28
1785,2022-09-29
1786,2022-09-30
1787,2022-10-01


In [11]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-11-09,0.007780,-0.114295,0.134691,0.007780,0.007780,0.303346,0.303346,0.303346,0.003491,0.003491,0.003491,0.299855,0.299855,0.299855,0.0,0.0,0.0,0.010140
1,2017-11-10,0.017105,-0.092609,0.136456,0.017105,0.017105,0.299194,0.299194,0.299194,-0.000111,-0.000111,-0.000111,0.299305,0.299305,0.299305,0.0,0.0,0.0,0.022223
2,2017-11-11,0.026430,-0.090913,0.154801,0.026430,0.026430,0.301369,0.301369,0.301369,0.003770,0.003770,0.003770,0.297599,0.297599,0.297599,0.0,0.0,0.0,0.034396
3,2017-11-12,0.035755,-0.082982,0.168440,0.035755,0.035755,0.302705,0.302705,0.302705,0.008090,0.008090,0.008090,0.294615,0.294615,0.294615,0.0,0.0,0.0,0.046579
4,2017-11-13,0.045080,-0.062341,0.182388,0.045080,0.045080,0.286095,0.286095,0.286095,-0.004157,-0.004157,-0.004157,0.290252,0.290252,0.290252,0.0,0.0,0.0,0.057978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2022-09-27,0.268637,0.217165,0.452886,0.268637,0.268637,0.251198,0.251198,0.251198,-0.008990,-0.008990,-0.008990,0.260188,0.260188,0.260188,0.0,0.0,0.0,0.336118
1784,2022-09-28,0.264643,0.212186,0.455730,0.264643,0.264643,0.261920,0.261920,0.261920,-0.002093,-0.002093,-0.002093,0.264013,0.264013,0.264013,0.0,0.0,0.0,0.333958
1785,2022-09-29,0.260649,0.207717,0.442929,0.260649,0.260649,0.272206,0.272206,0.272206,0.003491,0.003491,0.003491,0.268716,0.268716,0.268716,0.0,0.0,0.0,0.331599
1786,2022-09-30,0.256655,0.208170,0.453206,0.256655,0.256655,0.273951,0.273951,0.273951,-0.000111,-0.000111,-0.000111,0.274061,0.274061,0.274061,0.0,0.0,0.0,0.326965


In [12]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
1783,2022-09-27,0.217165,0.452886,0.336118
1784,2022-09-28,0.212186,0.455730,0.333958
1785,2022-09-29,0.207717,0.442929,0.331599
1786,2022-09-30,0.208170,0.453206,0.326965
1787,2022-10-01,0.205799,0.455392,0.324311


In [13]:
# Salvar modelo
with open('model_ada_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)